In [ ]:
!pip install openai

In [ ]:
!pip install python-dotenv

In [ ]:
cd '/content/drive/MyDrive/Courses/AI/Lecture14'

/content/drive/MyDrive/Courses/AI/Lecture14


In [ ]:
import requests
import json

# Load JSON data from a file
with open('documents.json', 'r') as file:
    documents_raw = json.load(file)

documents_raw[0]['documents'][0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?'}

In [ ]:
documents = []
for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [ ]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [ ]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [ ]:
from collections import defaultdict

In [ ]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [ ]:
len(hashes), len(documents)

(947, 948)

In [ ]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [ ]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [ ]:
import json

In [ ]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [ ]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [ ]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record.

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [ ]:
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv('api_keys.env')
api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI
client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

In [ ]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [ ]:
from tqdm.auto import tqdm
import pickle

In [ ]:
results = {}

In [ ]:
for doc in tqdm(documents):
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

# Save the list to a pickle file
with open('results.pkl', 'wb') as f:
    pickle.dump(results, f)

  0%|          | 0/948 [00:00<?, ?it/s]

In [ ]:
file_path = 'results.pkl'
with open(file_path, 'rb') as file:
    results = pickle.load(file)


In [ ]:
results['1f6520ca']

'["What qualifications should I have before joining this class?", "Are there any requirement conditions to enroll in this program?", "Where can I find the essential experience needed to start this course?", "Is there a list of necessary skills I need to have before beginning this course?", "What prior knowledge should I have to prepare for this course?"]'

In [ ]:
import ast
import re
parsed_results = {}

for doc_id, json_questions in results.items():
    # Clean up the string to remove unwanted characters and line breaks
    cleaned_json_questions = re.sub(r"\s+", " ", json_questions.strip())

    try:
        # Parse cleaned JSON
        questions_list = json.loads(cleaned_json_questions)
        # Store formatted questions in parsed_results
        parsed_results[doc_id] = questions_list
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON for {doc_id}: {e}")
        parsed_results[doc_id] = []  #

Error parsing JSON for 58c9f99f: Invalid \escape: line 1 column 109 (char 108)
Error parsing JSON for c91ad8f2: Invalid \escape: line 1 column 184 (char 183)
Error parsing JSON for a3aa3a7d: Invalid \escape: line 1 column 187 (char 186)


In [ ]:
parsed_results

{'c02e79ef': ['When does the course officially begin?',
  'What is the starting date and time of the course?',
  'How can I subscribe to the course calendar?',
  'Where can I register before the course starts?',
  "How do I join the course's communication channels?"],
 '1f6520ca': ['What qualifications should I have before joining this class?',
  'Are there any requirement conditions to enroll in this program?',
  'Where can I find the essential experience needed to start this course?',
  'Is there a list of necessary skills I need to have before beginning this course?',
  'What prior knowledge should I have to prepare for this course?'],
 '7842b56a': ['Can I participate in this course if I miss the start date?',
  'Am I allowed to submit assignments if I register late for this course?',
  'If I join the course after it begins, will I still have submission deadlines?',
  'Is it possible to enroll in the course after it officially has started?',
  'Can I complete the final projects if I

In [ ]:
doc_index = {d['id']: d for d in documents}

In [ ]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [ ]:
df.to_csv('ground-truth-data.csv', index=False)

In [ ]:
!head ground-truth-data.csv

question,course,document
When does the course officially begin?,data-engineering-zoomcamp,c02e79ef
What is the starting date and time of the course?,data-engineering-zoomcamp,c02e79ef
How can I subscribe to the course calendar?,data-engineering-zoomcamp,c02e79ef
Where can I register before the course starts?,data-engineering-zoomcamp,c02e79ef
How do I join the course's communication channels?,data-engineering-zoomcamp,c02e79ef
What qualifications should I have before joining this class?,data-engineering-zoomcamp,1f6520ca
Are there any requirement conditions to enroll in this program?,data-engineering-zoomcamp,1f6520ca
Where can I find the essential experience needed to start this course?,data-engineering-zoomcamp,1f6520ca
Is there a list of necessary skills I need to have before beginning this course?,data-engineering-zoomcamp,1f6520ca
